# Epsylon-greedy Method

This notebook is was built by Camille-Amaury JUGE in order to better understands the epsylon-greedy RL method. 

We will construct an agent which will try to find its path in a matrix (labyrinth) without walls.

## Imports

In [173]:
import numpy as np
from random import randint
import random
import sys
import time

## Classes

In [174]:
class Labyrinth(object):
    def __init__(self, x, y, max_reward, punition, final_pos):
        super(Labyrinth, self).__init__()
        self.lab = np.array([np.array([punition for j in range(y)]) for i in range(x)])
        self.lab[final_pos[0]][final_pos[1]] = max_reward
        print(self.lab)
        
    def update_Pos(self, x, y, direction):
        if (y == 0 and direction == 0) or (y == self.lab.shape[1]-1 and direction == 1) or (x == 0 and direction == 2) or (x == self.lab.shape[0]-1 and direction == 3):
            pass
        else:
            y = y + (-1 if direction == 0 else (1 if direction == 1 else 0))
            x = x + (-1 if direction == 2 else (1 if direction == 3 else 0))
        return (x,y)
        
        

In [175]:
class Robot(object):
    def __init__(self, init_pos, final_pos, lab, experiencing_rate, decreasing_exp_rate, learning_rate):
        super(Robot, self).__init__()
        self.init_pos = init_pos
        self.pos = self.init_pos
        self.final_pos = final_pos
        self.actions = ["Left", "Right", "Top", "Bottom"]
        self.labyrinth = lab
        self.Q = np.zeros((self.labyrinth.lab.shape[0], self.labyrinth.lab.shape[1], len(self.actions)))
        self.init_rates = [experiencing_rate, decreasing_exp_rate, learning_rate]
        self.rates = self.init_rates
        
    def play(self, epoch, max_iteration_per_epoch):
        for i in range(epoch):
            exploring = (random.uniform(0, 1) < self.rates[0])
            self.history = []
            self.pos = self.init_pos
            iteration = 0
            
            while((self.pos != self.final_pos) and (iteration < max_iteration_per_epoch)):
                action = self.choose_action(exploring)
                if action == "Left":
                    self.history.append((self.pos[0], self.pos[1], 0))
                    self.pos = self.labyrinth.update_Pos(self.pos[0], self.pos[1], 0)
                if action == "Right":
                    self.history.append((self.pos[0], self.pos[1], 1))
                    self.pos = self.labyrinth.update_Pos(self.pos[0], self.pos[1], 1)
                if action == "Top":
                    self.history.append((self.pos[0], self.pos[1], 2))
                    self.pos = self.labyrinth.update_Pos(self.pos[0], self.pos[1], 2)
                if action == "Bottom":
                    self.history.append((self.pos[0], self.pos[1], 3))
                    self.pos = self.labyrinth.update_Pos(self.pos[0], self.pos[1], 3)
                iteration += 1
                
                
            if self.pos != self.final_pos:
                self.history.append((self.final_pos[0],self.final_pos[1],0))
                
            self.update_Q()
            self.rates[0] = self.rates[0] - self.rates[1]
            
        
    def update_Q(self):
        sum_reward = 0
        for i, pos in enumerate(self.history):
            if i != 0:
                sum_reward += self.labyrinth.lab[pos[0]][pos[1]]
        
        weight_update = self.init_rates[2] * sum_reward
        for i, pos in enumerate(self.history):
            self.Q[pos[0]][pos[1]][pos[2]] += weight_update
        
                
        
                
        
    def choose_action(self, exploring):
        # if experiencing then random decision
        if exploring:
            return self.actions[randint(0, 1)]
        # if using memory then maximize rewards
        else:
            possibilities = self.Q[self.pos[0]][self.pos[1]]
            # find max action
            max_score = -100000
            max_i = -1
            for i, score in enumerate(possibilities):
                if score > max_score:
                    max_i = i
                    max_score = score
            return self.actions[max_i]
        
    def __repr__(self):
        return self.__str__()
    def __str__(self):
        s = ""
        for i, rows in enumerate(self.Q):
            s += "|"
            for j, column in enumerate(rows):
                if self.final_pos == (i,j):
                    s+="🞬|"
                else:
                    max_score = -10000000
                    max_k = -1
                    for k, score in enumerate(column):
                        if score > max_score:
                            max_k = k
                            max_score = score
                    if max_k == 0:
                        s += "🡄|"
                    elif max_k == 1:
                        s += "🡆|"
                    elif max_k == 2:
                        s += "🡅|"
                    elif max_k == 3:
                        s += "🡇|"
            s += "\n"
        return s
                

## Process

In [176]:
_final_position = (0,0)
_init_position = (3,9)
_lab_dim = (4,10)
_rewards = (50,-1)
# epsilon rate
_experiencing_rate = 0.99
_decreasing_exp_rate = 0.000005
_learning_rate = 0.0001
_epochs = 20000
_max_iteration_random = 100

In [177]:
lab_1 = Labyrinth(_lab_dim[0], _lab_dim[1], _rewards[0], _rewards[1],_final_position)
robot_1 = Robot(_init_position, _final_position, lab_1, _experiencing_rate, _decreasing_exp_rate, _learning_rate)

[[50 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [178]:
time1 = time.time()
robot_1.play(_epochs, _max_iteration_random)
time2 = time.time()
print('function took {:.3f} ms'.format((time2-time1)*1000.0))

function took 10287.480 ms


## Interpretation

how to interpret :

Arrows represent the way the agent tried to maximize the reward by reaching the X cross.

In [179]:
print(robot_1)

|🞬|🡄|🡄|🡄|🡄|🡄|🡄|🡄|🡄|🡄|
|🡅|🡅|🡄|🡅|🡇|🡅|🡇|🡇|🡅|🡅|
|🡇|🡅|🡄|🡄|🡄|🡄|🡄|🡄|🡄|🡄|
|🡇|🡅|🡅|🡇|🡇|🡅|🡅|🡇|🡇|🡅|

